*Technical University of Munich<br>
Professorship of Environmental Sensing and Modeling<br><br>*
**Author:**  Daniel Kühbacher<br>
**Date:**  03.09.2024

--- 

# Calculate Motorway CO emissions at different speed limits

<!--Notebook description and usage information-->
This notebook implements the <utls/hot_emission_process.py> function and multiprocessing to calculate hot vehicle emissions for a given area. 


In [1]:
# import libraries
import sys
import os
os.environ['USE_PYGEOS'] = '0'

import multiprocessing
import geopandas as gpd
import pandas as pd
from datetime import datetime

sys.path.append('../../utils')
import data_paths
from traffic_counts import TrafficCounts
from hbefa_hot_emissions import HbefaHotEmissions
from hot_emission_process import process_daily_emissions, process_hourly_emissions

# Reload local modules on changes
%reload_ext autoreload
%autoreload 2

# Notebook Settings

In [13]:
# Define start and end time for emission calculation. Ideally this should cover a whole year.
year = 2019
start_date = datetime(year, 1, 1)
end_date = datetime(year, 12, 31)

# define filename of the visum file
visum_filename = "visum_links.GPKG"

# if True, the script will only calculate the emission for the area within the roi polygon
clip_to_area = True
roi_polygon = data_paths.MUNICH_BOARDERS_FILE # defines ROI for clipping

# select aggregated or los-specific mode for traffic situation calculation
#mode = 'aggregated' 
mode = 'los_specific'

vehicle_classes = ['PC', 'LCV', 'HGV', 'BUS', 'MOT']
components = ['HC', 'CO', 'NOx', 'PM', 'PN', 'CO2(rep)',
              'CO2(total)', 'NO2', 'CH4', 'NMHC', 'PM (non-exhaust)', 'Benzene', 'PM2.5',
              'BC (exhaust)', 'PM2.5 (non-exhaust)', 'BC (non-exhaust)', 'CO2e']

components = ['CO', 'CO2(total)']

# Choose emission type: Tank-To-Wheel, Well-To-Tank (WTT), Well-To-Wheel (WTW)
# WTW includes upstream emisssions from fuel production and distribution
emission_type = 'EFA_weighted'
#emission_type = 'EFA_WTT_weighted'
#emission_type = 'EFA_WTW_weighted'

# if True, the timeprofiles for the selected components will be calculated
calculate_timeprofile = False
store_timeprofiles = False

# define number of processes for multiprocessing
NUMBER_OF_PROCESSES = 7

###
#
# STORE RESULTS
#
###

store_results = False
store_filename = f'linesource_Munich_{year}_los_specific.gpkg'

## Import Data and Initialize Objects

In [3]:
# import visum model
visum_raw = gpd.read_file(data_paths.VISUM_FOLDER_PATH + visum_filename)

if clip_to_area:
    roi = gpd.read_file(roi_polygon).to_crs(visum_raw.crs)
    visum = gpd.clip(visum_raw, roi)
    visum = visum.explode(ignore_index=True) # convert multipolygons to polygons

#visum = visum_links
visum = visum.reset_index(drop = True).reset_index() # reset index for calculation

# initialize traffic cycles
cycles = TrafficCounts()

In [14]:
visum_temp = visum.copy()

# change speed in VISUM model
#visum_temp.loc[visum_temp['road_type'].isin(['Motorway-Nat']) & (visum_temp['hbefa_speed'] == 120), 'hbefa_speed'] = '>130'

# subselect only motorway road links
visum_temp = visum_temp[visum_temp['road_type'].isin(['Motorway-Nat'])]

# initialize specific HBEFA emission factors
hbefa = HbefaHotEmissions(components= components, 
                          vehicle_classes= vehicle_classes, 
                          ef_type= emission_type, 
                          area_type= 'Urban',)

Loaded emission factors from /Users/daniel_tum/Documents/code/drive-inventory/data/restricted_input/hbefa/EFA_HOT_ts_hbefa.txt
Loaded emission factors from /Users/daniel_tum/Documents/code/drive-inventory/data/restricted_input/hbefa/EFA_HOT_aggregated_hbefa.txt


## Process Inventory
Use multiprocessing to calculate the emission for each road link day by day. This process will take some time to be finished for the whole area of interest.

In [15]:
# calculate emission for each day
dates = [d.strftime("%Y-%m-%d") for d in pd.date_range(start = start_date,
                                                       end = end_date,
                                                       freq = '1d')]

with multiprocessing.Manager() as manager: 
    
    result_queue = manager.Queue()
    error_queue = manager.Queue()
    
    with multiprocessing.Pool(NUMBER_OF_PROCESSES) as pool:
        parameters = [(d,
                       mode,
                       visum_temp.to_dict('records'),cycles,
                       hbefa,
                       result_queue,
                       error_queue,
                       ) for d in dates]
        
        res = pool.starmap(process_daily_emissions, parameters)
    
    # concatenate final process results.
    result = result_queue.get() #get first result from queue
    while not result_queue.empty():
        print('Concatenate final process results')
        new_result = result_queue.get()
        for road_index, emissions in result.items():
            for component, value in emissions.items():
                add_emissions = new_result[road_index][component]
                result[road_index][component] += add_emissions
                
    # retrieve process errors
    errors = list()
    while not error_queue.empty(): 
        errors.append(error_queue.get())

Finished calculating 2019-01-01
Finished calculating 2019-01-02
Finished calculating 2019-01-03
Finished calculating 2019-01-04
Finished calculating 2019-01-05
Finished calculating 2019-01-06
Finished calculating 2019-01-07
Finished calculating 2019-01-08
Finished calculating 2019-01-09
Finished calculating 2019-01-10
Finished calculating 2019-01-11
Finished calculating 2019-01-12
Finished calculating 2019-01-13
Finished calculating 2019-01-14
Finished calculating 2019-01-15
Finished calculating 2019-01-16
Finished calculating 2019-01-17
Finished calculating 2019-01-18
Finished calculating 2019-01-19
Finished calculating 2019-01-20
Finished calculating 2019-01-21
Finished calculating 2019-01-22
Finished calculating 2019-01-23
Finished calculating 2019-01-24
Finished calculating 2019-01-25
Finished calculating 2019-01-26
Finished calculating 2019-01-27
Finished calculating 2019-01-28
Finished calculating 2019-01-29
Finished calculating 2019-01-30
Finished calculating 2019-01-31
Finished

In [16]:
# print errors
for e in errors:
    print (e)

## Concatenate Results
All results are saved in result dict. This can be appended to the traffic model. 

In [17]:
# concatenate results and append to visum dataframe

result_df = pd.DataFrame(result).transpose()
result_df.columns = result_df.columns.map('_'.join)
visum_result = pd.concat([visum_temp, result_df], axis = 1)

## Calculate Emission Totals

In [18]:
visum_result['CO_total'] = visum_result[[v + '_CO' for v in vehicle_classes]].sum(axis=1) * visum_result['geometry'].length/1000
visum_result['CO2_total'] = visum_result[[v + '_CO2(total)' for v in vehicle_classes]].sum(axis=1) * visum_result['geometry'].length/1000

In [19]:
print(visum_result['CO_total'].sum() * 1e-6) # convert to tons
print(visum_result['CO2_total'].sum() * 1e-9) # convert to tons

1032.5465431238586
474.8817115187604


In [ ]:
474.8817115187604/1032.5465431238586

In [20]:
530000/3539.631601047075

149.73309647343476

All Motorway emissions with los-specific emission factors and urban area type:  CO = 1032.54 t <br>
All Motorway emissions with aggregated emission factors and rural area type: CO = 1223.27 t <br>



All Motorway emissions with aggregated emission factors and motorway area type: CO = 3539.63 t <br>

Total hot emission CO emissions for Munich are 2583 t <br>

All Motorway emissions with aggregated emission factors and urban area type: CO = 906.15 t <br>
All Motorway emissions with aggregated emission factors and rural area type: CO = 1223.27 <br>
All Motorway emissions with aggregated emission factors and motorway area type: CO = 3539.63 t <br>
CO2 = 530.54 kt